## transferlearning_003.ipynb

### Model 2 : 10%의 데이터 + Data Augmentation

In [1]:
from helper_functions import unzip_data, walk_through_dir, create_tensorboard_callback, plot_loss_curves

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 데이터 다운로드
!wget -q https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

In [4]:
# 10_food_classes_10_percent.zip 압축 해제
unzip_data("10_food_classes_10_percent.zip")

In [5]:
walk_through_dir("10_food_classes_10_percent")

10_food_classes_10_percent 폴더에는 2개의 폴더와 0개의 파일이 존재합니다.
10_food_classes_10_percent\test 폴더에는 10개의 폴더와 0개의 파일이 존재합니다.
10_food_classes_10_percent\test\chicken_curry 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent\test\chicken_wings 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent\test\fried_rice 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent\test\grilled_salmon 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent\test\hamburger 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent\test\ice_cream 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent\test\pizza 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent\test\ramen 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent\test\steak 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent\test\sushi 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent\train 폴더에는 10개의 폴더와 0개의 파일이 존재합니다.
10_food_classes_10_percent\train\chicken_curry 폴더에는 0개의 폴더와 75개의 파일이 존재합니다.
10_food_classes_10_percent\t

In [11]:
# 상수 선언
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCH_COUNT = 5

train_dir_10_percent = "10_food_classes_10_percent/train/"
test_dir_10_percent = "10_food_classes_10_percent/test/"

In [7]:
# 데이터셋 준비
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory = train_dir_10_percent,
    image_size = IMAGE_SIZE,
    label_mode = "categorical",
    batch_size = BATCH_SIZE
)

test_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory = test_dir_10_percent,
    image_size = IMAGE_SIZE,
    label_mode = "categorical",
    batch_size = BATCH_SIZE
)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [8]:
# Data Augmentation
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

data_augmentation = keras.Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2)
], name = "data_augmentation")

In [9]:
# Model 2 : 10%의 데이터 + Data Augmentation
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False

inputs = layers.Input(shape = input_shape, name = "model_2_input_layer")
x = data_augmentation(inputs)
x = base_model(x, training = False)
x = layers.GlobalAveragePooling2D(name = "model_2_global_average_pooling_layer")(x)
outputs = layers.Dense(10, activation = "softmax", name = "model_2_output_layer")(x)
model_2 = keras.Model(inputs, outputs)

model_2.compile(
    loss = "categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [10]:
# checkpoint 위치 지정
checkpoint_path = "10_percent_model_checkpoints_weights/checkpoint.ckpt"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    save_weights_only = True, # 전체 모델을 저장하려면 False로 지정
    save_best_only = False,  # 각 epoch의 모델 대신 최적화된 모델을 저장하려면 True로 지정
    save_freq = "epoch", # 매 epoch가 저장
)

In [12]:
history_10_percent_data_augment = model_2.fit(
    train_data_10_percent,
    epochs = EPOCH_COUNT,
    steps_per_epoch = len(train_data_10_percent),
    validation_data = test_data_10_percent,
    validation_steps = len(test_data_10_percent),
    callbacks = [
        create_tensorboard_callback("transfer_learning", "10_percent_data_augment"),
        checkpoint_callback
    ]
)

TensorBoard 로그 파일을 저장한 디렉토리 : transfer_learning/1_percent_data_augment/20210620-104203
c:\200210619_1\venv\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
Epoch 1/5
 4/24 [====>.........................] - ETA: 1:06 - loss: 2.4652 - accuracy: 0.0938

KeyboardInterrupt: 